In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import random
from modulator import Modulator
from demodulator import Demodulator

In [ ]:
def main(index: int):
    global berResult
    global noiseSNR
    global noiseResult
    global noiseTests
    global noneResult

    # LoRa Parameters
    Fs = 48000  # Sample rate [samples/s]
    BW = Fs // 2  # Bandwidth [Hz]
    fa = 0  # Start frequency [Hz]
    fb = BW  # Stop frequency [Hz]
    SF = 4  # Spread factor == Bits per symbol
    samples = 16384  # Number of samples

    # Modulator and demodulator
    mod = Modulator(samples, Fs, fa, fb, SF)
    demod = Demodulator(samples, Fs, fa, fb, SF)

    # Preamble and Data
    value = random.randint(0, 2**SF - 1)
    y_preamble = mod.getSignal([value])

    # Generate white noise
    noise_preamble = np.random.normal(
        0, noiseTests[index]+0.001, size=len(y_preamble)
    )

    # Calculate signal, noise power and SNR
    timeLength = 1 / len(y_preamble)
    sign = 10 * np.log10(np.sum(y_preamble**2) * timeLength)
    noise = 10 * np.log10(np.sum(noise_preamble**2) * timeLength)
    # print(
    #     "Noise Power: %.3f - Signal Power: %.3f - SNR: %.3f"
    #     % (noise, sign, sign - noise),
    # )
    noiseSNR[index] += sign - noise

    # Add noise
    y_preamble += noise_preamble

    # Separate signal in chunks of size "samples"
    leng = int(np.floor(len(y_preamble) / samples))
    for index in range(leng):
        result = demod.detectSymbol(
            y_preamble[index * samples : samples * (index + 1)]
        )
        if result is not None:
            # Calculate Hamming distance
            r = (1 << np.arange(8))[:, None]
            ber = np.count_nonzero((result & r) != (value & r)) / SF
            # Add to BER result variable
            berResult += ber
        else:
            noneResult += 1
            berResult += 1.0


In [ ]:
# Number of tests
testN = 10
# BER Results
berResult = 0.0
noneResult = 0

# Noise setup
noiseTests = np.arange(16,17)
noiseSNR = [0] * len(noiseTests)
noiseResult = [0] * len(noiseTests)

# Run number of tests
for index in range(len(noiseTests)):
    berResult = 0.0
    noneResult = 0
    print("%3d" % (index), end=" - ")
    for i in range(testN):
        main(index)
    noiseResult[index] = berResult / testN * 100.0
    noiseSNR[index] = noiseSNR[index] / testN
    # Print results
    print(
        "SNR: %.4f dB, BER result: %.4f%%, None result: %d"
        % (noiseSNR[index], noiseResult[index], noneResult),
    )

plt.plot(noiseSNR, noiseResult)
